In [1]:
#! pip install -r ./requirements.txt

In [2]:
# If it's not already installed ...
# !pip install git+https://github.com/amazon-science/chronos-forecasting.git

In [3]:
import numpy as np
import pandas as pd
import torch
from chronos import ChronosPipeline
import warnings

warnings.filterwarnings('ignore')

In [4]:
pipeline = ChronosPipeline.from_pretrained("amazon/chronos-t5-tiny", device_map="mps")

In [5]:
df = pd.read_parquet("./sales.parquet")

In [6]:
history = df.groupby("store").head(-12)
future = df.groupby("store").tail(12)

In [7]:
# context must be either a 1D tensor, a list of 1D tensors,
# or a left-padded 2D tensor with batch as the first dimension
context = [torch.tensor(i) for i in history.groupby('store')['sales'].apply(list)]
prediction_length = 12
forecast = pipeline.predict(context, prediction_length)  # shape [num_series, num_samples, prediction_length]

In [8]:
low = []
median = []
high = []
for i, store in enumerate(forecast):
    p10, p50, p90 = np.quantile(forecast[i].numpy(), [0.1, 0.5, 0.9], axis=0)
    low.extend(p10)
    median.extend(p50)
    high.extend(p90)

future["low"] = low
future["median"] = median
future["high"] = high

In [9]:
future

,store,week,sales,low,median,high
131,1,2012-08-10,1592409.97,1.513289e+06,1.613644e+06,1.690049e+06
132,1,2012-08-17,1597868.05,1.481359e+06,1.539519e+06,1.597678e+06
133,1,2012-08-24,1494122.38,1.391269e+06,1.482499e+06,1.550923e+06
134,1,2012-08-31,1582083.40,1.446006e+06,1.550923e+06,1.653557e+06
135,1,2012-09-07,1661767.33,1.482499e+06,1.568028e+06,1.690049e+06
...,...,...,...,...,...,...
6430,45,2012-09-28,713173.95,7.186146e+05,7.684940e+05,8.137338e+05
6431,45,2012-10-05,733455.07,7.116544e+05,7.887940e+05,8.317140e+05
6432,45,2012-10-12,734464.36,7.133943e+05,7.742940e+05,8.705733e+05
6433,45,2012-10-19,718125.53,7.348541e+05,7.858942e+05,8.943531e+05
